<a href="https://colab.research.google.com/github/pinzger/handsonllms/blob/main/Tokens%20and%20Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tokens and Embeddings
Exploring tokenizers and embeddings as an integral part of building LLMs. Examples adopted from Chapter 2 of [Hands-On Large Language Models](https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961).

---

💡 **NOTE**: For using a GPU in Google Colab, go to
**Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**.

---

If you are viewing this notebook on Google Colab (or any other cloud vendor), you might need to **uncomment and run** the following codeblock to install the dependencies for this chapter:

In [ ]:
# %%capture
# !pip install transformers>=4.41.2 sentence-transformers>=3.0.1 gensim>=4.3.2 scikit-learn>=1.5.0 accelerate>=0.31.0

# Using a tokenizer

Note that we load the model and tokenizer separately and keep them as such so that we can explore them separately.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [ ]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|>"
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
print(input_ids)


{'input_ids': tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
           293, 16423,   292,   286,   728,   481, 29889, 12027,  7420,   920,
           372,  9559, 29889, 32001]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


In [ ]:
print(tokenizer.decode(input_ids["input_ids"][0]))

Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|>


In [ ]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|>"

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate the text
output_ids = model.generate(
  input_ids=input_ids["input_ids"],
  max_new_tokens=100,
  num_beams=5,
  early_stopping=True
)

# Print the output
print(tokenizer.decode(output_ids[0]))

Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|> Subject: Sincere Apologies for the Gardening Mishap


Dear Sarah,


I hope this message finds you well. I am writing to express my deepest apologies for the unfortunate incident that occurred in your garden yesterday.


As you know, I have always admired your green thumb and the lush oasis you've cultivated over the years. It was never my intention to cause any harm to your beloved plants


In [ ]:
print(output_ids[0])

tensor([14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
          293, 16423,   292,   286,   728,   481, 29889, 12027,  7420,   920,
          372,  9559, 29889, 32001,  3323,   622, 29901,   317,  3742,   406,
         6225, 11763,   363,   278, 19906,   292,   341,   728,   481,    13,
           13,    13, 29928,   799, 19235, 29892,    13,    13,    13, 29902,
         4966,   445,  2643, 14061,   366,  1532, 29889,   306,   626,  5007,
          304,  4653,   590,  6483,   342,  3095, 11763,   363,   278,   443,
         6477,   403, 15134,   393, 10761,   297,   596, 16423, 22600, 29889,
           13,    13,    13,  2887,   366,  1073, 29892,   306,   505,  2337,
         7336,  2859,   596,  7933, 28968,   322,   278,   301,  1878,   288,
        25101,   366, 29915,   345, 18834,   630,   975,   278,  2440, 29889,
          739,   471,  2360,   590, 16392,   304,  4556,   738, 10311,   304,
          596,  1339,  8238, 18577], device='cuda:0')


In [ ]:
print(tokenizer.decode(14350))
print(tokenizer.decode(3323))
print(tokenizer.decode(622))
print(tokenizer.decode([3323, 622]))
print(tokenizer.decode(29901))

Write
Sub
ject
Subject
:


## Getting the embedding of a word from the trained LLM

In [ ]:
import torch

# Getting the embedding of a token
prompt = "germany"
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
print("Tokens: ", input_ids)
print(tokenizer.decode(input_ids["input_ids"][0][0]))
print(tokenizer.decode(input_ids["input_ids"][0][1]))

with torch.no_grad():
  outputs = model(**input_ids, output_hidden_states=True)

# Last hidden state
last_hidden_state = outputs.hidden_states[-1]  # Shape: [batch_size, seq_len, hidden_dim]

# Extract the embedding for the word (2 tokens)
embedding = last_hidden_state[0, :]

print(embedding)

# Different approach
# Extract embeddings for the first token (excluding special tokens)
# token_embeddings = last_hidden_state[:, 1:-1, :]  # Exclude [CLS] and [SEP] if present
# print(token_embeddings)

# Example: Get the embedding for the first word
#word_embedding = token_embeddings[0, 0, :]  # Shape: [hidden_dim]



Tokens:  {'input_ids': tensor([[22593,  1384]], device='cuda:0'), 'attention_mask': tensor([[1, 1]], device='cuda:0')}
germ
any
tensor([[ 0.3457,  0.6367,  0.6016,  ...,  2.3125,  1.0000,  0.4277],
        [-0.4668,  1.2812,  0.7500,  ...,  0.0289,  0.0267, -0.2812]],
       device='cuda:0', dtype=torch.bfloat16)


# Comparing Trained LLM Tokenizers


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
        )

In [ ]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "       "
12.0*50=600
"""

In [ ]:
show_tokens(text, "bert-base-uncased")

[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " / t / t " three tab ##s : " " 12 . 0 * 50 = 600 [SEP] 

In [ ]:
show_tokens(text, "bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " Three ta ##bs : " " 12 . 0 * 50 = 600 [SEP] 

In [ ]:
show_tokens(text, "gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]


 English  and  CAP ITAL IZ ATION 
 � � �  � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :"        "  Three  tabs :  "              " 
 12 . 0 * 50 = 600 
 

In [ ]:
show_tokens(text, "google/flan-t5-small")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

English and CA PI TAL IZ ATION  <unk>  <unk> show _ to ken s Fal s e None  e l if = = > = else : two tab s : " " Three tab s : " " 12. 0 * 50 = 600  </s> 

In [ ]:
# The official is `tiktoken` but this the same tokenizer on the HF platform
show_tokens(text, "Xenova/gpt-4")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



 English  and  CAPITAL IZATION 
 � � �  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         "
 12 . 0 * 50 = 600 
 

In [ ]:
# You need to request access before being able to use this tokenizer
show_tokens(text, "bigcode/starcoder2-15b")

tokenizer_config.json:   0%|          | 0.00/7.88k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]


 English  and  CAPITAL IZATION 
 � � �   � � 
 show _ tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

In [ ]:
show_tokens(text, "facebook/galactica-1.3b")

tokenizer_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.14M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]


 English  and  CAP ITAL IZATION 
 � � � �  � � � 
 show _ tokens  False  None  elif   ==   > =  else :  two  t abs : "      "  Three  t abs :   "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

In [ ]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<s>  
 English and C AP IT AL IZ ATION 
 � � � �  � � � 
 show _ to kens False None elif == >= else : two tabs :"    " Three tabs : "       " 
 1 2 . 0 * 5 0 = 6 0 0 
 

# Word Embeddings Beyond LLMs


In [ ]:
import gensim.downloader as api

# Download embeddings (66MB, glove, trained on wikipedia, vector size: 50)
# Other options include "word2vec-google-news-300"
# More options at https://github.com/RaRe-Technologies/gensim-data
model = api.load("glove-wiki-gigaword-50")

In [ ]:
print(model['king'])

[ 0.50451   0.68607  -0.59517  -0.022801  0.60046  -0.13498  -0.08813
  0.47377  -0.61798  -0.31012  -0.076666  1.493    -0.034189 -0.98173
  0.68229   0.81722  -0.51874  -0.31503  -0.55809   0.66421   0.1961
 -0.13495  -0.11476  -0.30344   0.41177  -2.223    -1.0756   -1.0783
 -0.34354   0.33505   1.9927   -0.04234  -0.64319   0.71125   0.49159
  0.16754   0.34344  -0.25663  -0.8523    0.1661    0.40102   1.1685
 -1.0137   -0.21585  -0.15155   0.78321  -0.91241  -1.6106   -0.64426
 -0.51042 ]


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

print(cosine_similarity([model['germany']], [model['berlin']]))
print(cosine_similarity([model['france']], [model['paris']]))

# Doing some arithmentic with the vectors
query = model['germany'] - model['berlin'] + model['france']
model.most_similar([query], topn=4)

[[0.7985422]]
[[0.802533]]


[('france', 0.9239919185638428),
 ('belgium', 0.83286052942276),
 ('spain', 0.7981604933738708),
 ('french', 0.7825075387954712)]

In [ ]:
# Doing the same with using positive and negative arguments
result = model.most_similar(positive=['germany', 'berlin'], negative=['france'], topn=10)
print(result)

[('warsaw', 0.7920182347297668), ('munich', 0.7543550729751587), ('dresden', 0.7494768500328064), ('prague', 0.7410918474197388), ('hamburg', 0.7180474996566772), ('vienna', 0.700072705745697), ('leipzig', 0.6986507773399353), ('bonn', 0.6699139475822449), ('frankfurt', 0.6662008166313171), ('heidelberg', 0.6654989719390869)]


In [ ]:
model.most_similar([model['paris']], topn=11)

[('paris', 1.0),
 ('prohertrib', 0.8611263036727905),
 ('france', 0.8025329113006592),
 ('brussels', 0.7796469926834106),
 ('amsterdam', 0.7769756317138672),
 ('french', 0.773611843585968),
 ('vienna', 0.7394115328788757),
 ('london', 0.7294350266456604),
 ('berlin', 0.7261149287223816),
 ('rome', 0.7099411487579346),
 ('strasbourg', 0.7078796029090881)]

In [ ]:
# Note, takes quite some time to download
modelw2v = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


# Recommending songs by embeddings

In [ ]:
import pandas as pd
from urllib import request

# Get the playlist dataset file
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

# Parse the playlist dataset file. Skip the first two lines as
# they only contain metadata
lines = data.read().decode("utf-8").split('\n')[2:]

# Remove playlists with only one song
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

# Load song metadata
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')

In [ ]:
print( 'Playlist #1:\n ', playlists[0], '\n')
print( 'Playlist #2:\n ', playlists[1])

Playlist #1:
  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '2', '42', '43', '44', '45', '46', '47', '48', '20', '49', '8', '50', '51', '52', '53', '54', '55', '56', '57', '25', '58', '59', '60', '61', '62', '3', '63', '64', '65', '66', '46', '47', '67', '2', '48', '68', '69', '70', '57', '50', '71', '72', '53', '73', '25', '74', '59', '20', '46', '75', '76', '77', '59', '20', '43'] 

Playlist #2:
  ['78', '79', '80', '3', '62', '81', '14', '82', '48', '83', '84', '17', '85', '86', '87', '88', '74', '89', '90', '91', '4', '73', '62', '92', '17', '53', '59', '93', '94', '51', '50', '27', '95', '48', '96', '97', '98', '99', '100', '57', '101', '102', '25', '103', '3', '104', '105', '106', '107', '47', '108', '109', '110', '111', '112', '113', '25', '63', '62', '114', '115', '84', '116', '117',

In [ ]:
from gensim.models import Word2Vec

# Train our Word2Vec model
model = Word2Vec(
    playlists, vector_size=32, window=20, negative=50, min_count=1, workers=4
)

In [ ]:
song_id = 2172

# Ask the model for songs similar to song #2172
model.wv.most_similar(positive=str(song_id))

[('2849', 0.9979680776596069),
 ('2640', 0.9964019060134888),
 ('3167', 0.9963980317115784),
 ('5549', 0.9959008693695068),
 ('2715', 0.9958351850509644),
 ('3117', 0.9954560995101929),
 ('2987', 0.9953479766845703),
 ('2881', 0.9951083660125732),
 ('2886', 0.9950577616691589),
 ('3094', 0.994985044002533)]

In [ ]:
print(songs_df.iloc[2172])

title     Fade To Black
artist        Metallica
Name: 2172 , dtype: object


In [ ]:
import numpy as np

def print_recommendations(song_id):
    similar_songs = np.array(
        model.wv.most_similar(positive=str(song_id),topn=5)
    )[:,0]
    return  songs_df.iloc[similar_songs]

# Extract recommendations
print_recommendations(2172)

,title,artist
id,,
2849,Run To The Hills,Iron Maiden
2640,Red Barchetta,Rush
3167,Unchained,Van Halen
5549,November Rain,Guns N' Roses
2715,Rainbow In The Dark,Dio


In [ ]:
print_recommendations(2172)

title     Fade To Black
artist        Metallica
Name: 2172 , dtype: object
['2849' '2640' '3167' '5549' '2715']


,title,artist
id,,
2849,Run To The Hills,Iron Maiden
2640,Red Barchetta,Rush
3167,Unchained,Van Halen
5549,November Rain,Guns N' Roses
2715,Rainbow In The Dark,Dio


In [ ]:
print_recommendations(842)

title     California Love (w\/ Dr. Dre & Roger Troutman)
artist                                              2Pac
Name: 842 , dtype: object
['5668' '413' '5661' '330' '886']


,title,artist
id,,
5668,How We Do (w\/ 50 Cent),The Game
413,If I Ruled The World (Imagine That) (w\/ Laury...,Nas
5661,Sweet Dreams,Beyonce
330,Hate It Or Love It (w\/ 50 Cent),The Game
886,Heartless,Kanye West
